In [7]:
import pymupdf 
import re

# 章节标题检测模式：匹配中文"第x章"或英文"Chapter x"格式
CHAPTER_PATTERN = r'^(第[\d一二三四五六七八九十]+章|Chapter\s+\d+)'

def detect_chapter(text):
    """
    检测文本是否为章节标题
    :param text: 待检测文本字符串
    :return: bool类型检测结果
    """
    if not text:  # 处理空输入
        return False
        
    # 使用正则表达式进行精确匹配
    return re.match(CHAPTER_PATTERN, text.strip()) is not None

def save_text_by_chapter(pdf_path, output_dir):
    # 打开 PDF 文件
    doc = pymupdf.open(pdf_path)
    chapter_text = ""
    chapter_count = 0
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        #print(text)
        # 假设每章以 第x章 开头
        
        if detect_chapter(text):
            chapter_count += 1
            chapter_text = text
        else:
            chapter_text += text
        
        # 保存章节文本到 txt 文件
        if chapter_count > 0:
            with open(f"{output_dir}/chapter_{chapter_count}.txt", "w", encoding="utf-8") as f:
                f.write(chapter_text)

# 使用示例
pdf_path = "buffet.pdf"
output_dir = "output"
save_text_by_chapter(pdf_path, output_dir)

In [ ]:
import os
import ollama
from pydantic import BaseModel
import json

class MetaKnolege(BaseModel):
    Declarative_Knowledge: list[str]
    Procedural_Knowledge: list[str]
    Conditional_Knowledge: list[str]
    
class MetaRegulation(BaseModel):
    Planning: list[str]
    Monitoring: list[str]
    Evaluating: list[str]
    
class MetaExperience(BaseModel):
    Positive: list[str]
    Negative: list[str]
    Neutral: list[str]
    
class MetaThink(BaseModel):
    MetaKnolege: MetaKnolege
    MetaRegulation: MetaRegulation
    MetaExperience: MetaExperience

class InvestmentBackground(BaseModel):
    ObjectToBeInvested: str
    Time: str
    BackgroundInfo: str

class InvestmentDecision(BaseModel):
    Time: str
    Investor: str
    Amount: str
    Object: str
    
class InvestProblem(BaseModel):
    MetaThink: MetaThink
    InvestmentBackground: InvestmentBackground
    InvestmentDecision: InvestmentDecision
    InvestmentReturn: str
    
def process_text_with_prompt(system_prompt, user_prompt):
    """使用ollama处理文本，支持system prompt和用户prompt的组合"""
    # 构造ollama需要的消息格式
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    # 调用ollama的chat接口
    response = ollama.chat(
        model='qwq',  # 根据实际模型调整，如'mistral'
        messages=messages,
        format=InvestProblem.model_json_schema(),
        options={"num_ctx":6000}
    )
    return response['message']['content']

def process_file_with_ollama(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        
        # 定义提示词
        system_prompt = """
        你是一名元认知科学家，按照如下的结构进行元认知分析：
        <struct>
            1. 元认知知识: 你对自己和他人在思考、解决问题和学习过程方面的了解。包括陈述性知识，程序性知识、条件性知识。
                1. 陈述性知识：关于自己和任务的事实，不包含任务本身的事实
                2. 程序性知识：知道如何执行策略
                3. 条件性知识：知道何时及为何使用策略
            2. 元认知调节: 你用来控制思维的活动和策略。包括规划、监控和评估。
                1. 规划：在开始任务前制定目标和选择策略。例如，决定在阅读前先浏览标题和问题以了解重点。
                2. 监控：在任务进行中检查进展和理解程度。例如，边读边问自己“我明白了吗？” 
                3. 评估：任务完成后反思结果和策略的有效性。例如，考试后回顾哪些复习方法最有用。
            3. 元认知体验: 学习新知识或尝试解决问题时产生的想法和感受。以下是其主要类型：
                1. 知道感（Feeling of Knowing, FOK）  
                    定义：在回忆信息前，觉得自己知道答案的一种感觉。  
                    示例：考试时，看到问题后觉得答案就在嘴边，但尚未完全回忆起来。  
                    作用：这种感觉可以指导是否继续努力回忆或转向其他策略。
                2. 舌尖现象（Tip-of-the-Tongue Experience, TOT）  
                    定义：知道某事但暂时无法回忆起来的感觉，通常伴随部分相关信息的激活。  
                    示例：试图回忆演员的名字，但只记得他们的电影角色。  
                    作用：这种体验提示记忆检索的接近性，可能促使个体通过提示或上下文辅助回忆。
                3. 信心判断（Confidence Judgments）  
                    定义：对自己答案或决定的确定程度评估。  
                    示例：在选择题中，感觉某个选项非常正确，决定不更改。  
                    作用：影响决策，如是否坚持答案或重新检查。
                4. 难度评估（Difficulty Assessments）  
                    定义：感知任务的难易程度或所需努力。  
                    示例：阅读复杂文章时，感觉需要更多时间理解，决定放慢速度。  
                    作用：帮助调整策略，如分解任务或寻求帮助。
                5. 情绪反应（Emotional Responses）  
                    定义：学习过程中伴随的情绪，如满足、挫折或焦虑。  
                    示例：掌握新概念后感到满足，或因反复失败感到挫折。  
                    作用：情绪反应可能触发策略调整，如因挫折而选择更简单的学习方法。
                6. 监控理解（Monitoring Comprehension）  
                    定义：在阅读或听讲时检查理解程度。  
                    示例：边读边问自己“我明白了吗？”发现不理解后重读。  
                    作用：实时调整学习过程，确保理解。
                7.自我提问（Self-Questioning）  
                    定义：通过提问检查理解或进展。  
                    示例：学习后问自己“这个概念的主要点是什么？”以确认掌握。  
                    作用：增强主动学习，识别知识缺口。
                8.反思性思考（Reflective Thinking）  
                    定义：任务完成后反思学习过程，评估策略效果。  
                    示例：考试后思考哪些复习方法有效，决定下次改进。  
                    作用：为未来学习提供反馈，优化方法。
        </struct>
        <example>
            元认知调节：
                规划：
                    从基本面开始分析。
                    在估值前讨论管理团队。
                监控：
                    考虑收购的金融环境。
                    反思综合成本费率的反直觉性。
                    关注管理团队的承保谨慎性变化。
                评估：
                    判断业务设置无特别之处。
                    对比1987年和1997年数据的可靠性。
                    评估投资价值并担忧管理层谨慎性。
            元认知知识：
                运用保险业务经验判断公司业绩。
                用巴菲特投资策略解释收购逻辑。

        </example>
        <example>
            元认知调节：
                规划：一个学生在准备数学考试时决定先复习公式，然后做练习题以测试掌握程度。  
                监控：在阅读科学文章时，学生发现自己不理解某段内容，于是停下来重读或查阅相关术语。  
                评估：考试后，学生反思发现用思维导图复习比单纯重读笔记更有效，决定下次继续使用该方法。
        </example>
        """
        user_prompt = f"""
        从下面案例中抽取出
        投资问题描述：
            1. 元认知：元认知知识、元认知调节和元认知体验。
            1. 投资背景描述：只需要包含客观内容，内容只需要包括待研究的投资对象，当时的时间，当时投资对象的背景信息,排除投资结论相关的所有内容。必须使用中立的语气描述。
            2. 最终投资决策：不要超过150字，包括时间、投资人、投资金额，投资对象。
            3. 最终投资回报：年化收益率，总收益额，投资年限
        <case>
            {content}
        </case>
        """
        #print(f"user_prompt: {user_prompt}")
        # 调用处理函数
        result = process_text_with_prompt(
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return result

# 遍历output目录里面的每一个文件
output_dir = "output"
results = []
for filename in os.listdir(output_dir):
    file_path = os.path.join(output_dir, filename)
    if os.path.isfile(file_path):
        result = process_file_with_ollama(file_path)
        print(f"Processed {filename}: {result}")
        results.append(result)
    #break

# 将结果写入文件
output_file = "corpus/output.json"
with open(output_file, "w+", encoding='utf-8') as f:
    for result in results:
        # format json string to one line
        result = json.dumps(result, ensure_ascii=False)
        f.write(f"{result}\n")

Processed chapter_6.txt: {
    "MetaThink": {
        "MetaKnolege": {
            "Declarative_Knowledge": [
                "了解杰克·瑞沃茨的经营哲学，即只在受挫或沮丧时考虑出售公司",
                "国民保险公司的业务特点：承保高风险但合理定价，拒绝无利润业务"
            ],
            "Procedural_Knowledge": [
                "评估保险公司价值时需分析已赚保费、净利润及浮存金规模",
                "运用市盈率和投资收益率等财务指标进行估值"
            ],
            "Conditional_Knowledge": [
                "当目标公司管理层有稳定经营哲学时，溢价收购可能获得长期收益",
                "在保险公司浮存金可自由投资且规模较大时优先考虑收购"
            ]
        },
        "MetaRegulation": {
            "Planning": [
                "先分析国民保险公司的财务数据和业务模式再进行估值",
                "比较市盈率与行业平均水平判断低估程度"
            ],
            "Monitoring": [
                "关注管理层承保谨慎性变化，如是否继续拒绝无利润业务",
                "跟踪浮存金投资组合的收益率波动情况"
            ],
            "Evaluating": [
                "对比收购价格（860万美元）与净资产价值（3200万+720万=3920万美元）验证低估程度",
                "通过1968年净利润增长评估管理团队能力提升"
            ]
        },
        "MetaExperience": {
            "Positiv